In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
import os 
import shutil
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt 
import torch
import cv2
from google.colab.patches import cv2_imshow
from cv2 import waitKey
import torch
import numpy as np
import time

#import pytesseract as pt 
%matplotlib inline

In [ ]:
filenames = []

size_props = {
    'height':[],
    'width':[]
}

bounding_box_props = {
    'xmin':[],
    'ymin':[],
    'xmax':[],
    'ymax':[]
}


annotations_path = 'annotations'
for file in tqdm(os.listdir(annotations_path)):
    annotation = ET.parse(os.path.join(annotations_path, file))
    filenames.append(os.path.join(annotations_path, file))
    size = annotation.find('size')
    for name, prop_list in size_props.items():
        prop_value = size.find(name).text
        size_props[name].append(int(prop_value))
    bounding_box = annotation.find('object').find('bndbox')
    for name, prop_list in bounding_box_props.items():
        prop_value = bounding_box.find(name).text
        bounding_box_props[name].append(int(prop_value))

In [ ]:
df = pd.DataFrame({
    'file':filenames,
    'width':size_props['width'],
    'height':size_props['height'],
    'xmin':bounding_box_props['xmin'],
    'ymin':bounding_box_props['ymin'],
    'xmax':bounding_box_props['xmax'],
    'ymax':bounding_box_props['ymax']
})

df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

df['bb_width'] = (df['xmax'] - df['xmin'])/df['width']
df['bb_height'] = (df['ymax'] - df['ymin'])/df['height']

In [ ]:
# Keeping important columns only 
yolo_df = df[['file', 'center_x', 'center_y', 'bb_width', 'bb_height']]
# Performing 70-15-15 split
test_size = int(0.15 * len(df))

df_train, df_test = train_test_split(yolo_df, test_size=test_size)
df_train, df_val = train_test_split(df_train, test_size=test_size)

In [ ]:
#setting the image path for train, test and validation

train_path = os.path.join('Images', 'train')
val_path = os.path.join('Images','val')
test_path = os.path.join('Images', 'test')
lp_path=os.path.join('Images', 'License_plate')
images_path = 'images'

if not os.path.exists(train_path):
    os.makedirs(train_path)
    print('Made folder for train set')

if not os.path.exists(val_path):
    os.makedirs(val_path)
    print('Made folder for val set')

if not os.path.exists(test_path):
    os.makedirs(test_path)
    print('Made folder for test set')
if not os.path.exists(lp_path):
    os.makedirs(lp_path)
    print('Made folder for lp set')

In [ ]:
#moving the images to train test and validation folder

print('Moving images for train set')
for _, row  in tqdm(df_train.iterrows()):
    annotation_path = row['file']
    image_name = os.path.split(annotation_path)[-1].replace('.xml','')
    image_src = os.path.join(images_path, f'{image_name}.png')
    image_dst = os.path.join(train_path, f'{image_name}.png')
    shutil.copy2(image_src, image_dst)
    label_text = f"0 {row['center_x']} {row['center_y']} {row['bb_width']} {row['bb_height']}"
    with open(os.path.join(train_path, f'{image_name}.txt'), 'w') as f:
        f.write(label_text)
print('Done moving images for train set')

print('Moving images for val set')
for _, row  in tqdm(df_val.iterrows()):
    annotation_path = row['file']
    image_name = os.path.split(annotation_path)[-1].replace('.xml','')
    image_src = os.path.join(images_path, f'{image_name}.png')
    image_dst = os.path.join(val_path, f'{image_name}.png')
    shutil.copy2(image_src, image_dst)
    label_text = f"0 {row['center_x']} {row['center_y']} {row['bb_width']} {row['bb_height']}"
    with open(os.path.join(val_path, f'{image_name}.txt'), 'w') as f:
        f.write(label_text)
print('Done moving images for val set')

print('Moving images for test set')
for _, row  in tqdm(df_test.iterrows()):
    annotation_path = row['file']
    image_name = os.path.split(annotation_path)[-1].replace('.xml','')
    image_src = os.path.join(images_path, f'{image_name}.png')
    image_dst = os.path.join(test_path, f'{image_name}.png')
    shutil.copy2(image_src, image_dst)
    label_text = f"0 {row['center_x']} {row['center_y']} {row['bb_width']} {row['bb_height']}"
    with open(os.path.join(test_path, f'{image_name}.txt'), 'w') as f:
        f.write(label_text)
print('Done moving images for test set')

In [ ]:
# Cloning the ultralytics yolo repository
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
!pip install -r yolov5/requirements.txt

In [ ]:
data = {
    'names':['License Plate'],
    'nc':1,
    'train':os.path.abspath(train_path),
    'val':os.path.abspath(val_path)
}

with open('data.yaml', 'w') as f:
    yaml.dump(data, f)

In [ ]:
!python ./yolov5/train.py --data ./data.yaml  --batch-size 8  --epochs 100 --weights yolov5/yolov5s.pt

In [ ]:
# Fetching the latest runs
yolo_path = 'yolov5/runs/train'
latest_run = os.listdir(yolo_path)[-1]

# Fetching the best weights 
best_weights = os.path.join(yolo_path, latest_run, 'weights', 'best.pt')

# Loading the model with best weights trained on custom data 
model = torch.hub.load('ultralytics/yolov5', 'custom', best_weights)

In [ ]:
def build_tesseract_options(psm=7):
		# tell Tesseract to only OCR alphanumeric characters
		alphanumeric = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
		options = "-c tessedit_char_whitelist={}".format(alphanumeric)
		# set the PSM mode
		options += " --psm {}".format(psm)
		# return the built options string
		return options

In [ ]:
#using OCR for license plate number extraction


video_path = r"TEST.mp4"  #input video path
cpu_or_cuda = "cuda"  #choose device; "cpu" or "cuda"(if cuda is available)
device = torch.device(cpu_or_cuda)
model = model.to(device)
frame = cv2.VideoCapture(video_path)

frame_width = int(frame.get(3))
frame_height = int(frame.get(4))
size = (frame_width, frame_height)
writer = cv2.VideoWriter('output_1.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)

text_font = cv2.FONT_HERSHEY_PLAIN
color= (0,255,0)
text_font_scale = 1.25
prev_frame_time = 0
new_frame_time = 0
k = 0
# Inference Loop
while True:
    ret, image = frame.read()
    if ret:
        output = model(image)
        result = np.array(output.pandas().xyxy[0])
        j=0
        for i in result:
            p1 = (int(i[0]),int(i[1]))
            p2 = (int(i[2]),int(i[3]))
            text_origin = (int(i[0]),int(i[1])-5)
            number_plate = image[int(i[1]):int(i[3]),int(i[0]):int(i[2])]
            cv2.imwrite('Number_plate/frame'+str(k)+'_'+str(j)+'.png', number_plate)
            options = build_tesseract_options(psm=7)

            text = pytesseract.image_to_string(number_plate,config=options)

            cv2.rectangle(image,p1,p2,color=color,thickness=4)  #drawing bounding boxes
            cv2.putText(image,text=f"{i[-1]} {i[-3]:.2f}"+"  "+ text,org=text_origin,
                        fontFace=text_font,fontScale=text_font_scale,
                        color=color,thickness=4)  #class and confidence text
            j=j+1
        new_frame_time = time.time()

        fps = 1/(new_frame_time-prev_frame_time)
        prev_frame_time = new_frame_time
        fps = int(fps)
        fps = str(fps)
        cv2.putText(image, fps, (7, 70), text_font, 3, (100, 255, 0), 3, cv2.LINE_AA)
        writer.write(image)
        cv2.imwrite('video_frame1/frame'+str(k)+'.png', image)

        cv2_imshow(image)
  
    else:
        break

    if waitKey(1) & 0xFF == ord('q'):
        break
    k=k+1

frame.release()
cv2.destroyAllWindows()